In [1]:
pip install tensorflow numpy matplotlib


In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers,models
import matplotlib.pyplot as plt

IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
NUM_CLASSES = 5

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
).flow_from_directory(
   "Documents/5th Sem/Machine Learning/ML Project/faceShape.v4i.folder/train",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode = 'categorical'
)

valid_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    "Documents/5th Sem/Machine Learning/ML Project/faceShape.v4i.folder/valid",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

base_model = MobileNetV2(
    input_shape=(128,128,3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = True
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable=False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=20,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

loss, acc = model.evaluate(valid_gen)
print(f"Validation Accuracy:{acc:.2f}")

model.save("face_shape_classifier_mobilenetv2.h5")
    
                      
    

Found 14362 images belonging to 5 classes.
Found 932 images belonging to 5 classes.
Epoch 1/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 622s 1s/step - accuracy: 0.2432 - loss: 1.9040 - val_accuracy: 0.2285 - val_loss: 1.7258
Epoch 2/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 784s 2s/step - accuracy: 0.3047 - loss: 1.6047 - val_accuracy: 0.2908 - val_loss: 1.5723
Epoch 3/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 504s 1s/step - accuracy: 0.3602 - loss: 1.5001 - val_accuracy: 0.3519 - val_loss: 1.4879
Epoch 4/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 511s 1s/step - accuracy: 0.3962 - loss: 1.4249 - val_accuracy: 0.3906 - val_loss: 1.4152
Epoch 5/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 502s 1s/step - accuracy: 0.4136 - loss: 1.3840 - val_accuracy: 0.4013 - val_loss: 1.3937
Epoch 6/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 462s 1s/step - accuracy: 0.4525 - loss: 1.3184 - val_accuracy: 0.4281 - val_loss: 1.3427
Epoch 7/20
449/449 ━━━━━━━━━━━━━━━━━━━━ 494s 1s/step - accuracy: 0.4793 - loss: 1.2791 - val_accuracy: 0.4388 - val_loss: 1.3067
Epoch 8/20
44

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img = image.load_img("./Beyonce_Face_shape_2.webp", target_size=IMAGE_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
predicted_class = list(train_gen.class_indices.keys())[np.argmax(prediction)]

print(f"Predicted face shape: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Predicted face shape: long
